# Weather Data

Data is sourced from NOAA, through their free [Climate Data Online](https://www.ncdc.noaa.gov/cdo-web/) service.

In [27]:
import csv

with open('sources/weather.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
  
header = data[0]
data = data[1:]

print(header)

['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09']


This includes data from the following stations:

| Station ID | City |
| --- | --- |
| USW00023174 | Los Angeles |
| USW00026451 | Anchorage |
| USW00014739 | Boston |

Each station reports the following data:

* Average Wind
* Precipitation
* Average Temperature

In [28]:
from dataclasses import dataclass

@dataclass
class Report:
    station: str
    date: str
    wind: float
    precip: float
    temp: float

In [29]:
reports = {}

for row in data:
    report = Report(
        row[header.index("STATION")],
        row[header.index("DATE")],
        float(row[header.index("AWND")]), # average wind
        float(row[header.index("PRCP")]), # precipitation
        float(row[header.index("TMAX")]) # max temp
    )
    if report.station not in reports:
        reports[report.station] = []
    reports[report.station].append(report)

print(*[f"{station}: {len(reports[station])} reports" for station in reports], sep='\n')

USW00023174: 365 reports
USW00026451: 365 reports
USW00014739: 365 reports


We'll want to map data to a scale. We can generate lookup tables (index -> note number) for a few common ones.

In [48]:
import itertools

def extend(scale):
  return list(note for note in itertools.chain(*([i + offset for offset in scale] for i in range(0, 127, 12))) if note in range(0, 127))

scales = {
  "chromatic": list(range(127)),
  "major": extend([0, 2, 4, 5, 7, 9, 11]),
  "minor": extend([0, 2, 3, 5, 7, 8, 10]),
  "pentatonic": extend([0, 2, 4, 7, 9]),
}

print(scales)

{'chromatic': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126], 'major': [0, 2, 4, 5, 7, 9, 11, 12, 14, 16, 17, 19, 21, 23, 24, 26, 28, 29, 31, 33, 35, 36, 38, 40, 41, 43, 45, 47, 48, 50, 52, 53, 55, 57, 59, 60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77, 79, 81, 83, 84, 86, 88, 89, 91, 93, 95, 96, 98, 100, 101, 103, 105, 107, 108, 110, 112, 113, 115, 117, 119, 120, 122, 124, 125], 'minor': [0, 2, 3, 5, 7, 8, 10, 12, 14, 15, 17, 19, 20, 22, 24, 26, 27, 29, 31, 32, 34, 36, 38, 39, 41, 43, 44, 46, 48, 50, 51, 53, 55, 56, 58

Audio is generated as MIDI files by traversing chronologically through the readings.

Temperature maps to note pitch, using a chromatic scale.

Wind speed maps to the modulation wheel.

In [67]:
import os
import mido

os.makedirs("output", exist_ok=True)
file = mido.MidiFile()

for station, station_reports in reports.items():
    track = mido.MidiTrack()

    file.tracks.append(track)

    track.append(mido.Message('program_change', program=0, time=0))
    track.append(mido.MetaMessage('track_name', name=station, time=0))
    
    prev_note = None

    for report in station_reports:
        note = scales["pentatonic"][int(round(report.temp * 3/5 - 8))]
        
        track.append(mido.Message('note_on', note=note, velocity=64, time=30))
        if prev_note and prev_note != note:
            track.append(mido.Message('note_off', note=prev_note, velocity=64, time=10))
        
        track.append(mido.Message('control_change', control=1, value=int(round(report.wind * 3)), time=0))
        track.append(mido.Message('control_change', control=21, value=int(round(report.precip * 30)), time=0))
        
        prev_note = note

file.save(f'output/weather.mid')